In [0]:
spark

In [0]:
import re
import pandas as pd
import numpy as np

In [0]:
# Make sure there are no extra spaces or characters in your key
storage_account_name = "retailstoragev1"
container_name = "rawdatav1"
storage_account_key = "PhgCS7e6Sis0d8hNKmiChdVoCUB+zyzJsZyfitbldYuGl2gbe0WnHU/s9avvn74sZOuwBVaBsZ64+AStORDGQw=="  # Replace with your actual key

# Clear any previous configurations (important!)
spark.conf.unset(f"fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net")
spark.conf.unset(f"fs.azure.account.oauth.provider.type.{storage_account_name}.dfs.core.windows.net")
spark.conf.unset(f"fs.azure.account.oauth2.client.id.{storage_account_name}.dfs.core.windows.net")

# Set the key configuration
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_account_key)

# Try accessing with explicit print statements
print("Attempting to connect to storage...")

try:
    result = dbutils.fs.ls(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/")
    print("Connection successful!")
    print(f"Found {len(result)} items")
    display(result)
except Exception as e:
    print(f"Error occurred: {str(e)}")
    
print("Code execution completed")

Attempting to connect to storage...
Connection successful!
Found 6 items


path,name,size,modificationTime
abfss://rawdatav1@retailstoragev1.dfs.core.windows.net/customers.csv,customers.csv,216754731,1742448678000
abfss://rawdatav1@retailstoragev1.dfs.core.windows.net/discounts.csv,discounts.csv,18275,1742439834000
abfss://rawdatav1@retailstoragev1.dfs.core.windows.net/employees.csv,employees.csv,15165,1742439834000
abfss://rawdatav1@retailstoragev1.dfs.core.windows.net/products.csv,products.csv,22028532,1742448708000
abfss://rawdatav1@retailstoragev1.dfs.core.windows.net/stores.csv,stores.csv,3105,1742448725000
abfss://rawdatav1@retailstoragev1.dfs.core.windows.net/transactions.csv,transactions.csv,805152362,1742440147000


Code execution completed


In [0]:
# Read the CSV file
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("abfss://rawdatav1@retailstoragev1.dfs.core.windows.net/customers.csv")

# Display the first few rows
df.show()

+-----------+--------------------+--------------------+--------------------+--------+-------------+------+-------------+--------------------+
|Customer ID|                Name|               Email|           Telephone|    City|      Country|Gender|Date Of Birth|           Job Title|
+-----------+--------------------+--------------------+--------------------+--------+-------------+------+-------------+--------------------+
|          1|        Tyler Garcia|tyler.garcia@fake...|  922.970.2265x47563|New York|United States|     M|   2003-07-15|                NULL|
|          2|       Joshua Miller|joshua.miller@fak...|     +1-958-729-6169|New York|United States|     M|   2000-06-16|     Records manager|
|          3| Alison Marshall DDS|alison.marshall.d...|+1-645-567-0876x5409|New York|United States|     F|   2003-07-22|                NULL|
|          4|      Jeffery Acosta|jeffery.acosta@fa...|  212.336.0912x84994|New York|United States|     M|   1996-11-12|         Proofreader|
|     

In [0]:
#print datatypes of columns
df.printSchema()

root
 |-- Customer ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Telephone: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Date Of Birth: date (nullable = true)
 |-- Job Title: string (nullable = true)



In [0]:
# Get number of rows
num_rows = df.count()

# Get number of columns
num_cols = len(df.columns)

# Print the shape
print(f"DataFrame Shape: ({num_rows}, {num_cols})")

DataFrame Shape: (1643306, 9)


In [0]:
from pyspark.sql.functions import *
#telephone column has lots of corrupt records
df.select(col("Telephone")).show()

+--------------------+
|           Telephone|
+--------------------+
|  922.970.2265x47563|
|     +1-958-729-6169|
|+1-645-567-0876x5409|
|  212.336.0912x84994|
|          7814535781|
|        469.238.0828|
|     +1-294-543-6438|
|   205.335.5074x1759|
|+1-436-359-4105x4...|
|+1-325-528-1530x0...|
|    325-379-6960x933|
|  (841)850-4720x0041|
|   (859)776-6705x615|
|        354.559.8541|
|          7359516571|
|    420.510.1425x994|
|    925.672.7219x347|
|          9492316332|
|   927-209-4313x3292|
|        375.997.2367|
+--------------------+
only showing top 20 rows



In [0]:
#check all unique countries in dataframe
df.select(col('Country')).distinct().show()

+--------------+
|       Country|
+--------------+
| United States|
|          中国|
|   Deutschland|
|United Kingdom|
|        España|
|        France|
|      Portugal|
+--------------+



In [0]:
#we will create separate dataframes for customers from different countries 
customer_usa_df = df.filter(col('country') == 'United States')
customer_deu_df = df.filter(col('country') == 'Deutschland')
customer_uk_df = df.filter(col('country') == 'United Kingdom')
customer_esp_df = df.filter(col('country') == 'España')
customer_fra_df = df.filter(col('country') == 'France')
customer_por_df = df.filter(col('country') == 'Portugal')
customer_chn_df = df.filter(col('country') == '中国')

In [0]:
#lets process phone numbers country wise 
customer_usa_df.select(col('Telephone')).show()

+--------------------+
|           Telephone|
+--------------------+
|  922.970.2265x47563|
|     +1-958-729-6169|
|+1-645-567-0876x5409|
|  212.336.0912x84994|
|          7814535781|
|        469.238.0828|
|     +1-294-543-6438|
|   205.335.5074x1759|
|+1-436-359-4105x4...|
|+1-325-528-1530x0...|
|    325-379-6960x933|
|  (841)850-4720x0041|
|   (859)776-6705x615|
|        354.559.8541|
|          7359516571|
|    420.510.1425x994|
|    925.672.7219x347|
|          9492316332|
|   927-209-4313x3292|
|        375.997.2367|
+--------------------+
only showing top 20 rows



In [0]:
customer_deu_df.select(col("Telephone")).show()

+-------------------+
|          Telephone|
+-------------------+
|   +49(0) 015449951|
|     (06626) 324077|
|   +49(0)1912 85838|
|+49 (0) 5531 612871|
|   +49(0)6523 20192|
|        02484053803|
|      (09692) 62978|
|         0275536574|
|   +49(0) 031979972|
|        07044 46172|
|  +49(0)4249 396487|
|     (04449) 731627|
|   +49(0)2665951453|
|        05468965668|
|       04667 325668|
|        01528570177|
|      (06507) 29774|
|   +49(0)7515 86564|
|      (00201) 77533|
|        01184 37536|
+-------------------+
only showing top 20 rows



In [0]:
def convert_to_number_format(phone_str):
    if not phone_str:
        return None
        
    # Remove all non-digit characters (spaces, dots, dashes, parentheses, extensions)
    digits_only = re.sub(r'\D', '', phone_str)
    
    # Handle US numbers with country code
    if len(digits_only) == 11:
        return digits_only[1:]  # Remove the leading '1'
    
    # Handle international format with +1
    if len(digits_only) > 11:
        # This might be a special case, return just the 10 digits after +1 if possible
        return digits_only[1:11] if len(digits_only) >= 11 else None
    
    # If it's already 10 digits, return as is
    if len(digits_only) == 10:
        return digits_only
    
    # If the number doesn't conform to expected patterns, return None
    return None
# Register UDF
number_format_udf = udf(convert_to_number_format, StringType())

# Apply the number format conversion
customer_deu_df = customer_deu_df.withColumn("Telephone", number_format_udf(col("Telephone")))

# Show results
customer_deu_df.show()


+-----------+--------------------+--------------------+----------+------+-----------+------+-------------+--------------------+
|Customer ID|                Name|               Email| Telephone|  City|    Country|Gender|Date Of Birth|           Job Title|
+-----------+--------------------+--------------------+----------+------+-----------+------+-------------+--------------------+
|     694533|           Mina Ring|mina.ring@fake_ya...|9001544995|Berlin|Deutschland|     F|   1996-04-21|Therapist, speech...|
|     694534|     Karl-Otto Butte|karl-otto.butte@f...|6626324077|Berlin|Deutschland|     M|   1995-02-12|Local government ...|
|     694535|     Anatoli Reising|anatoli.reising@f...|9019128583|Berlin|Deutschland|     M|   1995-10-25|           Economist|
|     694536|Frau Sofie Rose MBA.|frau.sofie.rose.m...|9055316128|Berlin|Deutschland|     F|   1983-09-01|      Teacher, music|
|     694537|       Valeri Biggen|valeri.biggen@fak...|9065232019|Berlin|Deutschland|     F|   1981-06-2

In [0]:
#similarly apply transformation to numbers of other countries 
customer_fra_df = customer_fra_df.withColumn("Telephone", number_format_udf(col("Telephone")))
customer_esp_df = customer_esp_df.withColumn("Telephone", number_format_udf(col("Telephone")))
customer_chn_df = customer_chn_df.withColumn("Telephone", number_format_udf(col("Telephone")))
customer_por_df = customer_por_df.withColumn("Telephone", number_format_udf(col("Telephone")))
customer_uk_df = customer_uk_df.withColumn("Telephone", number_format_udf(col("Telephone")))




In [0]:
#replace any missing value to 'Phone Number Unknown'
customer_fra_df = customer_fra_df.withColumn('Telephone', when(customer_fra_df['Telephone'].isNull(), "Phone Number Unknown").otherwise(customer_fra_df['Telephone']))
customer_esp_df = customer_esp_df.withColumn('Telephone', when(customer_esp_df['Telephone'].isNull(), "Phone Number Unknown").otherwise(customer_esp_df['Telephone']))
customer_deu_df = customer_deu_df.withColumn('Telephone', when(customer_deu_df['Telephone'].isNull(), "Phone Number Unknown").otherwise(customer_deu_df['Telephone']))
customer_chn_df = customer_chn_df.withColumn('Telephone', when(customer_chn_df['Telephone'].isNull(), "Phone Number Unknown").otherwise(customer_chn_df['Telephone']))
customer_uk_df = customer_uk_df.withColumn('Telephone', when(customer_uk_df['Telephone'].isNull(), "Phone Number Unknown").otherwise(customer_uk_df['Telephone']))
customer_por_df = customer_por_df.withColumn('Telephone', when(customer_por_df['Telephone'].isNull(), "Phone Number Unknown").otherwise(customer_por_df['Telephone']))




In [0]:
# do same transformation for USA
customer_usa_df = customer_usa_df.withColumn("Telephone", number_format_udf(col("Telephone")))
customer_usa_df = customer_usa_df.withColumn('Telephone', when(customer_usa_df['Telephone'].isNull(), "Phone Number Unknown").otherwise(customer_usa_df['Telephone']))

#drop telephonme column
customer_usa_df = customer_usa_df.drop('Telephonme')

#show USA dataframe
customer_usa_df.show()


+-----------+--------------------+--------------------+----------+--------+-------------+------+-------------+--------------------+
|Customer ID|                Name|               Email| Telephone|    City|      Country|Gender|Date Of Birth|           Job Title|
+-----------+--------------------+--------------------+----------+--------+-------------+------+-------------+--------------------+
|          1|        Tyler Garcia|tyler.garcia@fake...|2297022654|New York|United States|     M|   2003-07-15|                NULL|
|          2|       Joshua Miller|joshua.miller@fak...|9587296169|New York|United States|     M|   2000-06-16|     Records manager|
|          3| Alison Marshall DDS|alison.marshall.d...|6455670876|New York|United States|     F|   2003-07-22|                NULL|
|          4|      Jeffery Acosta|jeffery.acosta@fa...|1233609128|New York|United States|     M|   1996-11-12|         Proofreader|
|          5|      Ashley Sanders|ashley.sanders@fa...|7814535781|New York|U

In [0]:
from pyspark.sql import functions as f

combined_df = customer_usa_df.union(customer_deu_df).union(customer_uk_df).union(customer_esp_df).union(customer_fra_df).union(customer_por_df).union(customer_chn_df)

combined_df.select(col('Country')).distinct().show()

+--------------+
|       Country|
+--------------+
| United States|
|   Deutschland|
|United Kingdom|
|        España|
|        France|
|      Portugal|
|          中国|
+--------------+



In [0]:
#replacing 中国 with China
combined_df = combined_df.withColumn('Country', when(combined_df['Country'] == '中国', 'China').otherwise(combined_df['Country']))


combined_df.select(col('Country')).distinct().show()

+--------------+
|       Country|
+--------------+
| United States|
|   Deutschland|
|United Kingdom|
|        España|
|        France|
|      Portugal|
|         China|
+--------------+



In [0]:
#replacing null values in job with 'Unknown'
combined_df = combined_df.withColumn('Job Title', when(combined_df['Job Title'].isNull(), "Unknown").otherwise(combined_df['Job Title']))


#total null values in each column
null_counts = combined_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in combined_df.columns])

# Show the result
null_counts.show()

+-----------+----+-----+---------+----+-------+------+-------------+---------+
|Customer ID|Name|Email|Telephone|City|Country|Gender|Date Of Birth|Job Title|
+-----------+----+-----+---------+----+-------+------+-------------+---------+
|          0|   0|    0|        0|   0|      0|     0|            0|        0|
+-----------+----+-----+---------+----+-------+------+-------------+---------+



In [0]:
customers_df = combined_df

In [0]:
# Make sure there are no extra spaces or characters in your key
storage_account_name = "retailstoragev1"
container_name = "transformedcustomersv1"
storage_account_key = "PhgCS7e6Sis0d8hNKmiChdVoCUB+zyzJsZyfitbldYuGl2gbe0WnHU/s9avvn74sZOuwBVaBsZ64+AStORDGQw=="  # Replace with your actual key

# Clear any previous configurations (important!)
spark.conf.unset(f"fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net")
spark.conf.unset(f"fs.azure.account.oauth.provider.type.{storage_account_name}.dfs.core.windows.net")
spark.conf.unset(f"fs.azure.account.oauth2.client.id.{storage_account_name}.dfs.core.windows.net")

# Set the key configuration
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_account_key)

# Try accessing with explicit print statements
print("Attempting to connect to storage...")

try:
    result = dbutils.fs.ls(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/")
    print("Connection successful!")
    print(f"Found {len(result)} items")
    display(result)
except Exception as e:
    print(f"Error occurred: {str(e)}")
    
print("Code execution completed")

Attempting to connect to storage...
Connection successful!
Found 6 items


path,name,size,modificationTime
abfss://transformedcustomersv1@retailstoragev1.dfs.core.windows.net/transformedcustomersv1/,transformedcustomersv1/,0,1742762564000
abfss://transformedcustomersv1@retailstoragev1.dfs.core.windows.net/transformeddiscountsv1/,transformeddiscountsv1/,0,1742763167000
abfss://transformedcustomersv1@retailstoragev1.dfs.core.windows.net/transformedemployeesv1/,transformedemployeesv1/,0,1742763161000
abfss://transformedcustomersv1@retailstoragev1.dfs.core.windows.net/transformedproductsv1/,transformedproductsv1/,0,1742763172000
abfss://transformedcustomersv1@retailstoragev1.dfs.core.windows.net/transformedstoresv1/,transformedstoresv1/,0,1742762992000
abfss://transformedcustomersv1@retailstoragev1.dfs.core.windows.net/transformedtransactionsv1/,transformedtransactionsv1/,0,1742762777000


Code execution completed


In [0]:
# Define the container and storage account names
storage_account_name = "retailstoragev1"
container_name = "transformedcustomersv1"

# Define the output path with the customers folder
output_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/transformedcustomersv1/"

# Write the DataFrame to ADLS Gen2, partitioned by the 'Country' column
customers_df.write \
    .option("header", "true") \
        .partitionBy("Country") \
            .mode("overwrite") \
                .csv(output_path)

In [0]:
# Make sure there are no extra spaces or characters in your key
storage_account_name = "retailstoragev1"
container_name = "rawdatav1"
storage_account_key = "PhgCS7e6Sis0d8hNKmiChdVoCUB+zyzJsZyfitbldYuGl2gbe0WnHU/s9avvn74sZOuwBVaBsZ64+AStORDGQw=="  # Replace with your actual key

# Clear any previous configurations (important!)
spark.conf.unset(f"fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net")
spark.conf.unset(f"fs.azure.account.oauth.provider.type.{storage_account_name}.dfs.core.windows.net")
spark.conf.unset(f"fs.azure.account.oauth2.client.id.{storage_account_name}.dfs.core.windows.net")

# Set the key configuration
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_account_key)

# Try accessing with explicit print statements
print("Attempting to connect to storage...")

try:
    result = dbutils.fs.ls(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/")
    print("Connection successful!")
    print(f"Found {len(result)} items")
    display(result)
except Exception as e:
    print(f"Error occurred: {str(e)}")
    
print("Code execution completed")

Attempting to connect to storage...
Connection successful!
Found 6 items


path,name,size,modificationTime
abfss://rawdatav1@retailstoragev1.dfs.core.windows.net/customers.csv,customers.csv,216754731,1742448678000
abfss://rawdatav1@retailstoragev1.dfs.core.windows.net/discounts.csv,discounts.csv,18275,1742439834000
abfss://rawdatav1@retailstoragev1.dfs.core.windows.net/employees.csv,employees.csv,15165,1742439834000
abfss://rawdatav1@retailstoragev1.dfs.core.windows.net/products.csv,products.csv,22028532,1742448708000
abfss://rawdatav1@retailstoragev1.dfs.core.windows.net/stores.csv,stores.csv,3105,1742448725000
abfss://rawdatav1@retailstoragev1.dfs.core.windows.net/transactions.csv,transactions.csv,805152362,1742440147000


Code execution completed


In [0]:
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("abfss://rawdatav1@retailstoragev1.dfs.core.windows.net/discounts.csv")

# Display the first few rows
df.show()

+----------+----------+-------+--------------------+---------+--------------------+
|     Start|       End|Discont|         Description| Category|        Sub Category|
+----------+----------+-------+--------------------+---------+--------------------+
|2020-01-01|2020-01-10|    0.4|40% discount duri...| Feminine|   Coats and Blazers|
|2020-01-01|2020-01-10|    0.4|40% discount duri...| Feminine|Sweaters and Knit...|
|2020-01-01|2020-01-10|    0.4|40% discount duri...|Masculine|   Coats and Blazers|
|2020-01-01|2020-01-10|    0.4|40% discount duri...|Masculine|Sweaters and Swea...|
|2020-01-01|2020-01-10|    0.4|40% discount duri...| Children|               Coats|
|2020-01-01|2020-01-10|    0.4|40% discount duri...| Children|            Sweaters|
|2020-03-15|2020-03-31|   0.35|35% discount duri...| Feminine|Dresses and Jumps...|
|2020-03-15|2020-03-31|   0.35|35% discount duri...| Feminine|  Shirts and Blouses|
|2020-03-15|2020-03-31|   0.35|35% discount duri...|Masculine|  T-shirts and

In [0]:
#display description 
df.select("Description").show(truncate=False)

+-------------------------------------------------------+
|Description                                            |
+-------------------------------------------------------+
|40% discount during our New Year Winter Sale           |
|40% discount during our New Year Winter Sale           |
|40% discount during our New Year Winter Sale           |
|40% discount during our New Year Winter Sale           |
|40% discount during our New Year Winter Sale           |
|40% discount during our New Year Winter Sale           |
|35% discount during our Early Spring Collection Refresh|
|35% discount during our Early Spring Collection Refresh|
|35% discount during our Early Spring Collection Refresh|
|35% discount during our Early Spring Collection Refresh|
|35% discount during our Early Spring Collection Refresh|
|25% discount during our Mid-Spring Refresh Sale        |
|25% discount during our Mid-Spring Refresh Sale        |
|25% discount during our Mid-Spring Refresh Sale        |
|25% discount 

In [0]:
#to fetch occasion from description

def occasion(description):
    return(" ".join( description.split()[4:6]))

occasion= udf(occasion, StringType())


In [0]:
#fetching occasion from description on the enttire column

df=df.withColumn("Occasion", occasion(df.Description))
df.show()

+----------+----------+-------+--------------------+---------+--------------------+------------------+
|     Start|       End|Discont|         Description| Category|        Sub Category|          Occasion|
+----------+----------+-------+--------------------+---------+--------------------+------------------+
|2020-01-01|2020-01-10|    0.4|40% discount duri...| Feminine|   Coats and Blazers|          New Year|
|2020-01-01|2020-01-10|    0.4|40% discount duri...| Feminine|Sweaters and Knit...|          New Year|
|2020-01-01|2020-01-10|    0.4|40% discount duri...|Masculine|   Coats and Blazers|          New Year|
|2020-01-01|2020-01-10|    0.4|40% discount duri...|Masculine|Sweaters and Swea...|          New Year|
|2020-01-01|2020-01-10|    0.4|40% discount duri...| Children|               Coats|          New Year|
|2020-01-01|2020-01-10|    0.4|40% discount duri...| Children|            Sweaters|          New Year|
|2020-03-15|2020-03-31|   0.35|35% discount duri...| Feminine|Dresses and

In [0]:
df=df.drop("Description")
df.show(3)

+----------+----------+-------+---------+--------------------+--------+
|     Start|       End|Discont| Category|        Sub Category|Occasion|
+----------+----------+-------+---------+--------------------+--------+
|2020-01-01|2020-01-10|    0.4| Feminine|   Coats and Blazers|New Year|
|2020-01-01|2020-01-10|    0.4| Feminine|Sweaters and Knit...|New Year|
|2020-01-01|2020-01-10|    0.4|Masculine|   Coats and Blazers|New Year|
+----------+----------+-------+---------+--------------------+--------+
only showing top 3 rows



In [0]:

null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()

+-----+---+-------+--------+------------+--------+
|Start|End|Discont|Category|Sub Category|Occasion|
+-----+---+-------+--------+------------+--------+
|    0|  0|      0|      10|          10|       0|
+-----+---+-------+--------+------------+--------+



In [0]:
#replacing category null values with unknown
df=df.withColumn("Category", when(col("Category").isNull(), "Unknown").otherwise(col("Category")))
df=df.withColumn("Sub Category", when(col("Sub Category").isNull(), "Unknown").otherwise(col("Sub Category")))
null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()

+-----+---+-------+--------+------------+--------+
|Start|End|Discont|Category|Sub Category|Occasion|
+-----+---+-------+--------+------------+--------+
|    0|  0|      0|       0|           0|       0|
+-----+---+-------+--------+------------+--------+



In [0]:
discounts_df=df
discounts_df.show(5)

+----------+----------+-------+---------+--------------------+--------+
|     Start|       End|Discont| Category|        Sub Category|Occasion|
+----------+----------+-------+---------+--------------------+--------+
|2020-01-01|2020-01-10|    0.4| Feminine|   Coats and Blazers|New Year|
|2020-01-01|2020-01-10|    0.4| Feminine|Sweaters and Knit...|New Year|
|2020-01-01|2020-01-10|    0.4|Masculine|   Coats and Blazers|New Year|
|2020-01-01|2020-01-10|    0.4|Masculine|Sweaters and Swea...|New Year|
|2020-01-01|2020-01-10|    0.4| Children|               Coats|New Year|
+----------+----------+-------+---------+--------------------+--------+
only showing top 5 rows



In [0]:
employee_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("abfss://rawdatav1@retailstoragev1.dfs.core.windows.net/employees.csv")

# Display the first few rows
employee_df.show()

+-----------+--------+--------------------+-----------------+
|Employee ID|Store ID|                Name|         Position|
+-----------+--------+--------------------+-----------------+
|          1|       1|     Stephen Johnson|    Store Manager|
|          2|       1|       Rebecca Myers|Assistant Manager|
|          3|       1|  Katherine Buchanan|          Cashier|
|          4|       1|       Jessica Hicks|      Stock Clerk|
|          5|       1|          Ryan Gross|  Sales Associate|
|          6|       1|     Jeffery Carlson|  Sales Associate|
|          7|       1|      Melissa Wilson|  Sales Associate|
|          8|       1|        Edward Hicks|  Sales Associate|
|          9|       1|        Shari Jordan|  Sales Associate|
|         10|       1|Mrs. Melissa Cald...|  Sales Associate|
|         11|       1|       Regina Nelson|  Sales Associate|
|         12|       1|        Kevin Dawson|  Sales Associate|
|         13|       1|       Nicole Murphy|  Sales Associate|
|       

In [0]:
#null value counts
null_counts = employee_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in employee_df.columns])
null_counts.show()

+-----------+--------+----+--------+
|Employee ID|Store ID|Name|Position|
+-----------+--------+----+--------+
|          0|       0|   0|       0|
+-----------+--------+----+--------+



In [0]:
products_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("abfss://rawdatav1@retailstoragev1.dfs.core.windows.net/products.csv")

# Display the first few rows
products_df.show(3,truncate=False)

+----------+--------+---------------------+---------------------------------+---------------------------------+-------------------------------------+----------------------------------+---------------------------------+------------------------+-----+--------+---------------+
|Product ID|Category|Sub Category         |Description PT                   |Description DE                   |Description FR                       |Description ES                    |Description EN                   |Description ZH          |Color|Sizes   |Production Cost|
+----------+--------+---------------------+---------------------------------+---------------------------------+-------------------------------------+----------------------------------+---------------------------------+------------------------+-----+--------+---------------+
|1         |Feminine|Coats and Blazers    |Esportivo Veludo Verde Com Botões|Sport Samt Sport Mit Knöpfen     |Sports Velvet Sports Avec Des Boutons|Deportes De Terciopelo Con

In [0]:
products_df.columns

['Product ID',
 'Category',
 'Sub Category',
 'Description PT',
 'Description DE',
 'Description FR',
 'Description ES',
 'Description EN',
 'Description ZH',
 'Color',
 'Sizes',
 'Production Cost']

In [0]:
products_df=products_df.select("Product ID", "Category", "Sub Category", "Description EN", "Color", "Sizes", "Production Cost")
products_df.show(3,truncate=False)

+----------+--------+---------------------+---------------------------------+-----+--------+---------------+
|Product ID|Category|Sub Category         |Description EN                   |Color|Sizes   |Production Cost|
+----------+--------+---------------------+---------------------------------+-----+--------+---------------+
|1         |Feminine|Coats and Blazers    |Sports Velvet Sports With Buttons|NULL |S|M|L|XL|10.73          |
|2         |Feminine|Sweaters and Knitwear|Luxurious Pink Denim With Buttons|PINK |S|M|L|XL|19.55          |
|3         |Feminine|Dresses and Jumpsuits|Black Tricot Printed Tricot      |BLACK|S|M|L|XL|25.59          |
+----------+--------+---------------------+---------------------------------+-----+--------+---------------+
only showing top 3 rows



In [0]:
#calculating null values for each columns
null_counts = products_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in products_df.columns]) 
null_counts.show()

+----------+--------+------------+--------------+-----+-----+---------------+
|Product ID|Category|Sub Category|Description EN|Color|Sizes|Production Cost|
+----------+--------+------------+--------------+-----+-----+---------------+
|         0|       0|           0|             0|49780| 8280|              0|
+----------+--------+------------+--------------+-----+-----+---------------+



In [0]:
#replacing null values with unknown

products_df=products_df.withColumn("Color", when(col("Color").isNull(), "Unknown").otherwise(col("Color")))

products_df=products_df.withColumn("Sizes", when(col("Sizes").isNull(), "Unknown").otherwise(col("Sizes")))

#calculating null values for each columns
null_counts = products_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in products_df.columns]) 
null_counts.show()

+----------+--------+------------+--------------+-----+-----+---------------+
|Product ID|Category|Sub Category|Description EN|Color|Sizes|Production Cost|
+----------+--------+------------+--------------+-----+-----+---------------+
|         0|       0|           0|             0|    0|    0|              0|
+----------+--------+------------+--------------+-----+-----+---------------+



In [0]:
products_df.show(3)

+----------+--------+--------------------+--------------------+-------+--------+---------------+
|Product ID|Category|        Sub Category|      Description EN|  Color|   Sizes|Production Cost|
+----------+--------+--------------------+--------------------+-------+--------+---------------+
|         1|Feminine|   Coats and Blazers|Sports Velvet Spo...|Unknown|S|M|L|XL|          10.73|
|         2|Feminine|Sweaters and Knit...|Luxurious Pink De...|   PINK|S|M|L|XL|          19.55|
|         3|Feminine|Dresses and Jumps...|Black Tricot Prin...|  BLACK|S|M|L|XL|          25.59|
+----------+--------+--------------------+--------------------+-------+--------+---------------+
only showing top 3 rows



In [0]:
products_df.select("Sizes").distinct().show()

+-----------------+
|            Sizes|
+-----------------+
|         P|M|G|GG|
|38|40|42|44|46|48|
|36|38|40|42|44|46|
|           M|L|XL|
|          Unknown|
|   38|40|42|44|46|
|      38|40|42|44|
|         S|M|L|XL|
|   36|38|40|42|44|
|       M|L|XL|XXL|
|            S|M|L|
|            P|M|G|
|      36|38|40|42|
+-----------------+



In [0]:
stores_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("abfss://rawdatav1@retailstoragev1.dfs.core.windows.net/stores.csv")

# Display the first few rows
stores_df.show()

+--------+--------------+-----------------+--------------------+-------------------+--------+--------+---------+
|Store ID|       Country|             City|          Store Name|Number of Employees|ZIP Code|Latitude|Longitude|
+--------+--------------+-----------------+--------------------+-------------------+--------+--------+---------+
|       1| United States|         New York|      Store New York|                 10|   10001| 40.7128|  -74.006|
|       2| United States|      Los Angeles|   Store Los Angeles|                  8|   90001| 34.0522|-118.2437|
|       3| United States|          Chicago|       Store Chicago|                  9|   60601| 41.8781| -87.6298|
|       4| United States|          Houston|       Store Houston|                 10|   77001| 29.7604| -95.3698|
|       5| United States|          Phoenix|       Store Phoenix|                  9|   85001| 33.4484| -112.074|
|       6|          中国|             上海|          Store 上海|                  8|  200000| 31.2304|

In [0]:
stores_df = stores_df.withColumn("Country", when(col("Country") == "中国", "China").otherwise(col('Country')))
stores_df.show()

+--------+--------------+-----------------+--------------------+-------------------+--------+--------+---------+
|Store ID|       Country|             City|          Store Name|Number of Employees|ZIP Code|Latitude|Longitude|
+--------+--------------+-----------------+--------------------+-------------------+--------+--------+---------+
|       1| United States|         New York|      Store New York|                 10|   10001| 40.7128|  -74.006|
|       2| United States|      Los Angeles|   Store Los Angeles|                  8|   90001| 34.0522|-118.2437|
|       3| United States|          Chicago|       Store Chicago|                  9|   60601| 41.8781| -87.6298|
|       4| United States|          Houston|       Store Houston|                 10|   77001| 29.7604| -95.3698|
|       5| United States|          Phoenix|       Store Phoenix|                  9|   85001| 33.4484| -112.074|
|       6|         China|             上海|          Store 上海|                  8|  200000| 31.230

In [0]:
# Extract Chinese city names as a Python list
chinese_cities = [row["City"] for row in stores_df.filter(col("Country") == "China").select("City").distinct().collect()]

# Print the extracted list
print(chinese_cities)


['深圳', '重庆', '上海', '北京', '广州']


In [0]:
null_counts = stores_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in stores_df.columns]) 
null_counts.show()

+--------+-------+----+----------+-------------------+--------+--------+---------+
|Store ID|Country|City|Store Name|Number of Employees|ZIP Code|Latitude|Longitude|
+--------+-------+----+----------+-------------------+--------+--------+---------+
|       0|      0|   0|         0|                  0|       0|       0|        0|
+--------+-------+----+----------+-------------------+--------+--------+---------+



In [0]:
pip install translate

  Obtaining dependency information for translate from https://files.pythonhosted.org/packages/e1/54/df0d3e62636f291d716caa44c5c9590af8e6f225c59d7874051da6428c06/translate-3.6.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for lxml from https://files.pythonhosted.org/packages/a5/73/762c319c4906b3db67e4abc7cfe7d66c34996edb6d0e8cb60f462954d662/lxml-5.3.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata
  Obtaining dependency information for libretranslatepy==2.1.1 from https://files.pythonhosted.org/packages/24/54/044ecbf4eeba3f3feee70d496f34f397dc20714cdb80dfa8bc01e7409f88/libretranslatepy-2.1.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/5.0 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/5.0 MB 2.5 MB/s eta 0:00:02
   ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/5.0 MB 1.8 MB/s eta 0:00:03
   ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/5.0 MB 3.0 MB/s eta 0:00:02
   ━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.7/5.0 MB 4.9

In [0]:
from translate import Translator
from pyspark.sql.functions import col, when, lit
from pyspark.sql.types import StringType

# Get distinct Chinese city names
chinese_cities = [row.City for row in stores_df.filter(col("Country") == "China").select("City").distinct().collect()]

# Create translation map without asyncio
city_translation_map = {}
for city in chinese_cities:
    try:
        translator = Translator(to_lang="en", from_lang="zh")
        translated_text = translator.translate(city)
        city_translation_map[city] = translated_text
    except Exception as e:
        print(f"Error translating {city}: {e}")
        # If translation fails, keep the original name
        city_translation_map[city] = city

# Create a mapping function
def translate_city(city):
    return city_translation_map.get(city, city)

# Register as UDF
translate_city_udf = udf(translate_city, StringType())

# Apply the translation
stores_df = stores_df.withColumn(
    "City",
    when(col("Country") == "China", translate_city_udf(col("City")))
    .otherwise(col("City"))
)

# Show results
stores_df.show()

+--------+--------------+-----------------+--------------------+-------------------+--------+--------+---------+
|Store ID|       Country|             City|          Store Name|Number of Employees|ZIP Code|Latitude|Longitude|
+--------+--------------+-----------------+--------------------+-------------------+--------+--------+---------+
|       1| United States|         New York|      Store New York|                 10|   10001| 40.7128|  -74.006|
|       2| United States|      Los Angeles|   Store Los Angeles|                  8|   90001| 34.0522|-118.2437|
|       3| United States|          Chicago|       Store Chicago|                  9|   60601| 41.8781| -87.6298|
|       4| United States|          Houston|       Store Houston|                 10|   77001| 29.7604| -95.3698|
|       5| United States|          Phoenix|       Store Phoenix|                  9|   85001| 33.4484| -112.074|
|       6|         China|         Shanghai|          Store 上海|                  8|  200000| 31.2

In [0]:
#drop store name column
stores_df = stores_df.drop(col("Store Name"))
stores_df.printSchema()

root
 |-- Store ID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Number of Employees: integer (nullable = true)
 |-- ZIP Code: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)



In [0]:
#load transactions data
transactions_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("abfss://rawdatav1@retailstoragev1.dfs.core.windows.net/transactions.csv")

# Display the first few rows
transactions_df.show()

+-------------------+----+-----------+----------+----+-------+----------+--------+-------------------+--------+----------+--------+-----------+--------+---------------+--------------------+----------------+--------------+-------------+
|         Invoice ID|Line|Customer ID|Product ID|Size|  Color|Unit Price|Quantity|               Date|Discount|Line Total|Store ID|Employee ID|Currency|Currency Symbol|                 SKU|Transaction Type|Payment Method|Invoice Total|
+-------------------+----+-----------+----------+----+-------+----------+--------+-------------------+--------+----------+--------+-----------+--------+---------------+--------------------+----------------+--------------+-------------+
|INV-US-001-03558761|   1|      47162|       485|   M|   NULL|      80.5|       1|2023-01-01 15:42:00|     0.0|      80.5|       1|          7|     USD|              $|          MASU485-M-|            Sale|          Cash|        126.7|
|INV-US-001-03558761|   2|      47162|      2779|   G|  

In [0]:
#check for null values
null_counts = transactions_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in transactions_df.columns])
null_counts.show()


+----------+----+-----------+----------+------+-------+----------+--------+----+--------+----------+--------+-----------+--------+---------------+---+----------------+--------------+-------------+
|Invoice ID|Line|Customer ID|Product ID|  Size|  Color|Unit Price|Quantity|Date|Discount|Line Total|Store ID|Employee ID|Currency|Currency Symbol|SKU|Transaction Type|Payment Method|Invoice Total|
+----------+----+-----------+----------+------+-------+----------+--------+----+--------+----------+--------+-----------+--------+---------------+---+----------------+--------------+-------------+
|         0|   0|          0|         0|413102|4350783|         0|       0|   0|       0|         0|       0|          0|       0|              0|  0|               0|             0|            0|
+----------+----+-----------+----------+------+-------+----------+--------+----+--------+----------+--------+-----------+--------+---------------+---+----------------+--------------+-------------+



In [0]:
transactions_df.select("Invoice ID").count()

6416827

In [0]:
#dropping Color column
transactions_df = transactions_df.drop(col("Color"))

In [0]:
#replacing null values in size col 
transactions_df = transactions_df.na.fill("Unknown")

In [0]:
#check for null values
null_counts = transactions_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in transactions_df.columns])
null_counts.show()


+----------+----+-----------+----------+----+----------+--------+----+--------+----------+--------+-----------+--------+---------------+---+----------------+--------------+-------------+
|Invoice ID|Line|Customer ID|Product ID|Size|Unit Price|Quantity|Date|Discount|Line Total|Store ID|Employee ID|Currency|Currency Symbol|SKU|Transaction Type|Payment Method|Invoice Total|
+----------+----+-----------+----------+----+----------+--------+----+--------+----------+--------+-----------+--------+---------------+---+----------------+--------------+-------------+
|         0|   0|          0|         0|   0|         0|       0|   0|       0|         0|       0|          0|       0|              0|  0|               0|             0|            0|
+----------+----+-----------+----------+----+----------+--------+----+--------+----------+--------+-----------+--------+---------------+---+----------------+--------------+-------------+



In [0]:
transactions_df.show(3)

+-------------------+----+-----------+----------+----+----------+--------+-------------------+--------+----------+--------+-----------+--------+---------------+----------------+----------------+--------------+-------------+
|         Invoice ID|Line|Customer ID|Product ID|Size|Unit Price|Quantity|               Date|Discount|Line Total|Store ID|Employee ID|Currency|Currency Symbol|             SKU|Transaction Type|Payment Method|Invoice Total|
+-------------------+----+-----------+----------+----+----------+--------+-------------------+--------+----------+--------+-----------+--------+---------------+----------------+----------------+--------------+-------------+
|INV-US-001-03558761|   1|      47162|       485|   M|      80.5|       1|2023-01-01 15:42:00|     0.0|      80.5|       1|          7|     USD|              $|      MASU485-M-|            Sale|          Cash|        126.7|
|INV-US-001-03558761|   2|      47162|      2779|   G|      31.5|       1|2023-01-01 15:42:00|     0.4| 

In [0]:
customers_df.printSchema() 

root
 |-- Customer ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Telephone: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Date Of Birth: date (nullable = true)
 |-- Job Title: string (nullable = true)



In [0]:
products_df.printSchema() 

root
 |-- Product ID: integer (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub Category: string (nullable = true)
 |-- Description EN: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Sizes: string (nullable = true)
 |-- Production Cost: double (nullable = true)



In [0]:
employee_df.printSchema()

root
 |-- Employee ID: integer (nullable = true)
 |-- Store ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Position: string (nullable = true)



In [0]:
discounts_df.printSchema()

root
 |-- Start: date (nullable = true)
 |-- End: date (nullable = true)
 |-- Discont: double (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub Category: string (nullable = true)
 |-- Occasion: string (nullable = true)



In [0]:
stores_df.printSchema()

root
 |-- Store ID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Number of Employees: integer (nullable = true)
 |-- ZIP Code: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)



In [0]:
employee_df.printSchema()

root
 |-- Employee ID: integer (nullable = true)
 |-- Store ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Position: string (nullable = true)



In [0]:
transactions_df.printSchema()

root
 |-- Invoice ID: string (nullable = false)
 |-- Line: integer (nullable = true)
 |-- Customer ID: integer (nullable = true)
 |-- Product ID: integer (nullable = true)
 |-- Size: string (nullable = false)
 |-- Unit Price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Line Total: double (nullable = true)
 |-- Store ID: integer (nullable = true)
 |-- Employee ID: integer (nullable = true)
 |-- Currency: string (nullable = false)
 |-- Currency Symbol: string (nullable = false)
 |-- SKU: string (nullable = false)
 |-- Transaction Type: string (nullable = false)
 |-- Payment Method: string (nullable = false)
 |-- Invoice Total: double (nullable = true)



In [0]:
cnt=transactions_df.select(col("Line")).distinct().show()
cnt

+----+
|Line|
+----+
|   1|
|   6|
|   3|
|   5|
|   4|
|   7|
|   2|
+----+



In [0]:
transactions_df.select(col("Currency")).distinct().show()

+--------+
|Currency|
+--------+
|     USD|
|     CNY|
|     EUR|
|     GBP|
+--------+



In [0]:
transactions_df.select(col("Currency Symbol")).distinct().show()
transactions_df=transactions_df.drop(col("Currency Symbol"))

+---------------+
|Currency Symbol|
+---------------+
|              $|
|              ¥|
|              €|
|              £|
+---------------+



In [0]:
transactions_df.printSchema()

root
 |-- Invoice ID: string (nullable = false)
 |-- Line: integer (nullable = true)
 |-- Customer ID: integer (nullable = true)
 |-- Product ID: integer (nullable = true)
 |-- Size: string (nullable = false)
 |-- Unit Price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Line Total: double (nullable = true)
 |-- Store ID: integer (nullable = true)
 |-- Employee ID: integer (nullable = true)
 |-- Currency: string (nullable = false)
 |-- SKU: string (nullable = false)
 |-- Transaction Type: string (nullable = false)
 |-- Payment Method: string (nullable = false)
 |-- Invoice Total: double (nullable = true)



In [0]:
joined_df = transactions_df.join(stores_df.select(col("Country"),col("Store ID")), on="Store ID", how="inner")

# Show the result
joined_df.show()

+--------+-------------------+----+-----------+----------+-------+----------+--------+-------------------+--------+----------+-----------+--------+--------------------+----------------+--------------+-------------+-------------+
|Store ID|         Invoice ID|Line|Customer ID|Product ID|   Size|Unit Price|Quantity|               Date|Discount|Line Total|Employee ID|Currency|                 SKU|Transaction Type|Payment Method|Invoice Total|      Country|
+--------+-------------------+----+-----------+----------+-------+----------+--------+-------------------+--------+----------+-----------+--------+--------------------+----------------+--------------+-------------+-------------+
|       1|INV-US-001-03558761|   1|      47162|       485|      M|      80.5|       1|2023-01-01 15:42:00|     0.0|      80.5|          7|     USD|          MASU485-M-|            Sale|          Cash|        126.7|United States|
|       1|INV-US-001-03558761|   2|      47162|      2779|      G|      31.5|       

In [0]:
joined_df.printSchema()

root
 |-- Store ID: integer (nullable = true)
 |-- Invoice ID: string (nullable = false)
 |-- Line: integer (nullable = true)
 |-- Customer ID: integer (nullable = true)
 |-- Product ID: integer (nullable = true)
 |-- Size: string (nullable = false)
 |-- Unit Price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Line Total: double (nullable = true)
 |-- Employee ID: integer (nullable = true)
 |-- Currency: string (nullable = false)
 |-- SKU: string (nullable = false)
 |-- Transaction Type: string (nullable = false)
 |-- Payment Method: string (nullable = false)
 |-- Invoice Total: double (nullable = true)
 |-- Country: string (nullable = true)



In [0]:
transactions_df=joined_df
# Make sure there are no extra spaces or characters in your key
storage_account_name = "retailstoragev1"
container_name = "transformedcustomersv1"
storage_account_key = "PhgCS7e6Sis0d8hNKmiChdVoCUB+zyzJsZyfitbldYuGl2gbe0WnHU/s9avvn74sZOuwBVaBsZ64+AStORDGQw=="  # Replace with your actual key

# Clear any previous configurations (important!)
spark.conf.unset(f"fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net")
spark.conf.unset(f"fs.azure.account.oauth.provider.type.{storage_account_name}.dfs.core.windows.net")
spark.conf.unset(f"fs.azure.account.oauth2.client.id.{storage_account_name}.dfs.core.windows.net")

# Set the key configuration
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_account_key)

# Try accessing with explicit print statements
print("Attempting to connect to storage...")

try:
    result = dbutils.fs.ls(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/")
    print("Connection successful!")
    print(f"Found {len(result)} items")
    display(result)
except Exception as e:
    print(f"Error occurred: {str(e)}")
    
print("Code execution completed")


Attempting to connect to storage...
Connection successful!
Found 1 items


path,name,size,modificationTime
abfss://transformedcustomersv1@retailstoragev1.dfs.core.windows.net/transformedcustomersv1/,transformedcustomersv1/,0,1742762564000


Code execution completed


In [0]:
# Define the output path
output_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/transformedtransactionsv1/"

# Write the DataFrame to ADLS Gen2, partitioned by the 'Country' column
transactions_df.write \
    .option("header", "true") \
        .partitionBy("Country") \
            .mode("overwrite") \
                .csv(output_path)

In [0]:
#write other dataframes to the destination folders
#stores table
output_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/transformedstoresv1/"
stores_df.write\
    .option("header", "true") \
    .mode("overwrite")\
        .csv(output_path)
#employees table
output_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/transformedemployeesv1/"
employee_df.write\
    .option("header", "true") \
    .mode("overwrite")\
        .csv(output_path)
#discounts table
output_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/transformeddiscountsv1/"
discounts_df.write\
    .option("header", "true") \
    .mode("overwrite")\
        .csv(output_path)
#products table
output_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/transformedproductsv1/"
products_df.write\
    .option("header", "true") \
    .mode("overwrite")\
        .csv(output_path)